In [51]:
import input_helpers
import numpy as np
import math
import cv2

In [159]:
places, rots, size = input_helpers.read_labels('/home/paperspace/Desktop/converted/car/training/suburu_pulling_to_left/suburu04/labels/250.pickle')

In [160]:
tmp = input_helpers.get_boxcorners(places, rots, size)
print (rots)
print (places)
print(size)

[ 0.076405]
[[ 5.57113  -3.024249 -0.806757]]
[[ 1.651   1.778   4.5212]]


In [161]:
print(tmp)
#tmp = tmp - places
#print(tmp)

[[[ 3.38498315 -4.08320854 -0.806757  ]
  [ 7.89299281 -3.73810225 -0.806757  ]
  [ 3.24926719 -2.31039575 -0.806757  ]
  [ 3.38498315 -4.08320854  0.844243  ]
  [ 3.24926719 -2.31039575  0.844243  ]
  [ 7.75727685 -1.96528946 -0.806757  ]
  [ 7.89299281 -3.73810225  0.844243  ]
  [ 7.75727685 -1.96528946  0.844243  ]]]


In [88]:
max_x, min_x = np.max(tmp[0, :, 0]), np.min(tmp[0, :, 0])
max_y, min_y = np.max(tmp[0, :, 1]), np.min(tmp[0, :, 1])

In [89]:
delta_x = max_x - min_x
delta_y = max_y - min_y

In [90]:
delta_x, delta_y

(4.6437256143709433, 2.1179190721718637)

In [84]:
angle = math.pi/2. - math.atan(delta_y / delta_x)
angle

1.1709010728165141

In [50]:
tmp2 = tmp[0, :][:, :2]
tmp2

array([[ 16.00224076,  -4.59153367],
       [ 20.52063053,  -4.43214918],
       [ 15.93956147,  -2.81463882],
       [ 16.00224076,  -4.59153367],
       [ 15.93956147,  -2.81463882],
       [ 20.45795124,  -2.65525433],
       [ 20.52063053,  -4.43214918],
       [ 20.45795124,  -2.65525433]])

In [64]:
tmp3 = np.reshape(cv2.convexHull(np.array(tmp2, dtype=np.float32)), (-1, 2))
tmp3

array([[ 20.52062988,  -4.43214941],
       [ 20.45795059,  -2.65525436],
       [ 15.93956184,  -2.81463885],
       [ 16.00224113,  -4.59153366]], dtype=float32)

In [66]:
max_x, min_x = np.max(tmp3[:, 0]), np.min(tmp3[:, 0])
max_y, min_y = np.max(tmp3[:, 1]), np.min(tmp3[:, 1])

In [93]:
delta_x_convx_hull = max_x - min_x
delta_y_convex_hull = max_y - min_y
delta_x_convx_hull, delta_y_convex_hull

(4.6437256143709433, 2.1179190721718637)

In [95]:
length_of_car_from_convex_hull = max([delta_x_convx_hull, delta_y_convex_hull])
width_of_car_from_convex_hull = min([delta_x_convx_hull, delta_y_convex_hull])
x_length_from_point = tmp[0][0]

In [125]:
print(tmp)

[[[-2.18614685 -1.05895954  0.        ]
  [ 2.32186281 -0.71385325  0.        ]
  [-2.32186281  0.71385325  0.        ]
  [-2.18614685 -1.05895954  1.651     ]
  [-2.32186281  0.71385325  1.651     ]
  [ 2.18614685  1.05895954  0.        ]
  [ 2.32186281 -0.71385325  1.651     ]
  [ 2.18614685  1.05895954  1.651     ]]]


In [126]:
x_dirn_diff = tmp[0][1][0] - tmp[0][2][0]
y_dirn_diff = tmp[0][5][1] - tmp[0][1][1]
print(x_dirn_diff, y_dirn_diff)

4.64372561437 1.77281278754


In [136]:
#a_mat = np.array([[length_of_car_from_convex_hull], [width_of_car_from_convex_hull]])
a_mat = np.array([length_of_car_from_convex_hull, width_of_car_from_convex_hull])
print(a_mat)
a_mat /= np.linalg.norm(a_mat)
#print(np.linalg.norm(a_mat))
#b_mat = np.array([[x_dirn_diff], [y_dirn_diff]])
b_mat = np.array([x_dirn_diff, y_dirn_diff])
print(b_mat)
b_mat /= np.linalg.norm(b_mat)
#right_pseudo_inverse = np.dot(np.transpose(a_mat), np.linalg.inv(np.dot(a_mat, np.transpose(a_mat))))
#r_mat = np.dot(b_mat, right_pseudo_inverse)
angle = math.acos(a_mat.dot(b_mat))
print(a_mat)
print(b_mat)
print(angle)
#print(r_mat)

[ 4.64372561  2.11791907]
[ 4.64372561  1.77281279]
[ 0.90983914  0.41496114]
[ 0.93423488  0.35665836]
0.06321152495831452


In [205]:
def get_orientation(corners_initial):
    average = np.sum(corners_initial, axis=0) / corners_initial.shape[0]
    print('average:', average)
    x_dirn_diff = corners_initial[1][0] - average[0]#corners[0][0]
    y_dirn_diff = corners_initial[2][1] - average[1]#corners[1][1]
    #x_dirn_diff = 4.50800965974
    #y_dirn_diff = 1.4277065029
    print ('x diff:', x_dirn_diff, 'y diff:', y_dirn_diff)
    corners = corners_initial - average
    print(corners)
    convx_hull = np.reshape(cv2.convexHull(np.array(corners[:, :2], dtype=np.float32)), (-1, 2))
    #print (cv2.boundingRect(np.array(corners[0, :][:, :2], dtype=np.float32)))
    print (convx_hull)
    max_x, min_x = np.max(convx_hull[:, 0]), np.min(convx_hull[:, 0])
    max_y, min_y = np.max(convx_hull[:, 1]), np.min(convx_hull[:, 1])
    print(min_x, max_x, min_y, max_y)
    delta_x_convx_hull = max_x - min_x
    delta_y_convex_hull = max_y - min_y
    length_of_car_from_convex_hull = max([delta_x_convx_hull, delta_y_convex_hull])
    width_of_car_from_convex_hull = min([delta_x_convx_hull, delta_y_convex_hull])
    print ('length of car:', length_of_car_from_convex_hull, 'width of car:', width_of_car_from_convex_hull)
    a_mat = np.array([length_of_car_from_convex_hull, width_of_car_from_convex_hull])
    a_mat /= np.linalg.norm(a_mat)
    print (a_mat)
    b_mat = np.array([x_dirn_diff, y_dirn_diff])
    b_mat /= np.linalg.norm(b_mat)
    print (b_mat)
    angle = math.acos(a_mat.dot(b_mat))
    print ('angle_diff:', np.degrees(angle))
    return angle

In [216]:
arrs = []
for i in range(1, 332):
    places, rots, size = input_helpers.read_labels('/home/paperspace/Desktop/converted/car/training/suburu_pulling_to_left/suburu04/labels/' + str(i) + '.pickle')
    tmp = input_helpers.get_boxcorners(places, rots, size)
    #print (rots)
    #print (places)
    #print(size)
    #print(tmp)
    arrs.append(get_orientation(tmp[0]))
    break

average: [ 56.239099  -0.423079  -1.0664  ]
x diff: 2.27654096869 y diff: 0.847345607098
[[-2.24389694 -0.9303547  -0.8255    ]
 [ 2.27654097 -0.84734561 -0.8255    ]
 [-2.27654097  0.84734561 -0.8255    ]
 [-2.24389694 -0.9303547   0.8255    ]
 [-2.27654097  0.84734561  0.8255    ]
 [ 2.24389694  0.9303547  -0.8255    ]
 [ 2.27654097 -0.84734561  0.8255    ]
 [ 2.24389694  0.9303547   0.8255    ]]
[[ 2.27654099 -0.84734559]
 [ 2.24389696  0.93035471]
 [-2.27654099  0.84734559]
 [-2.24389696 -0.93035471]]
-2.27654 2.27654 -0.930355 0.930355
length of car: 4.55308 width of car: 1.86071
[ 0.92568332  0.3782993 ]
[ 0.93718678  0.34882794]
angle_diff: 1.81276493651


In [214]:
print (np.degrees(sum(arrs) / len(arrs)))

4.01909709938


In [181]:
tmp = input_helpers.get_boxcorners(places, rots, size)
print (rots)
print (places)
print(size)
print(tmp)

[ 0.076405]
[[ 5.57113  -3.024249 -0.806757]]
[[ 1.651   1.778   4.5212]]
[[[ 3.38498315 -4.08320854 -0.806757  ]
  [ 7.89299281 -3.73810225 -0.806757  ]
  [ 3.24926719 -2.31039575 -0.806757  ]
  [ 3.38498315 -4.08320854  0.844243  ]
  [ 3.24926719 -2.31039575  0.844243  ]
  [ 7.75727685 -1.96528946 -0.806757  ]
  [ 7.89299281 -3.73810225  0.844243  ]
  [ 7.75727685 -1.96528946  0.844243  ]]]


In [280]:
def get_orientation(corners_initial):
    corners_initial = np.reshape(corners_initial, (-1, 3))
    average = np.sum(corners_initial, axis=0) / corners_initial.shape[0]
    corners = corners_initial - average
    #print(corners[:, :2])
    convx_hull = np.reshape(cv2.convexHull(np.array(corners[:, :2], dtype=np.float32)), (-1, 2))
    print (convx_hull)
    #max_x, min_x = np.max(convx_hull[:, 0]), np.min(convx_hull[:, 0])
    #max_y, min_y = np.max(convx_hull[:, 1]), np.min(convx_hull[:, 1])
    #print(min_x, max_x, min_y, max_y)
    #delta_x_convx_hull = max_x - min_x
    #delta_y_convex_hull = max_y - min_y
    rotated_convx_hull = np.vstack((convx_hull[1:], convx_hull[:1]))
    #print(rotated_convx_hull)
    distance_delta = convx_hull - rotated_convx_hull
    distance_delta = np.linalg.norm(convx_hull - rotated_convx_hull, axis=1)
    #print(distance_delta)
    length_of_car_from_convex_hull = np.max(distance_delta)
    width_of_car_from_convex_hull = np.min(distance_delta)
    #print ('length of car:', length_of_car_from_convex_hull, 'width of car:', width_of_car_from_convex_hull)
    l, w = length_of_car_from_convex_hull, width_of_car_from_convex_hull
    a_mat = [[-l / 2., -w / 2.],
            [l / 2., -w / 2.],
            [-l / 2., w / 2.],
            [-l / 2., -w / 2.],
            [-l / 2., w / 2.],
            [l / 2., w / 2.],
            [l / 2., -w / 2.],
            [l / 2., w / 2.]]
    b_mat = corners[:, :2]
    #print (a_mat)
    #print (b_mat)
    rot_mat = np.linalg.lstsq(a_mat, b_mat)[0]
    #print(rot_mat)
    angle = math.acos(rot_mat[0][0])
    #print ('angle_diff:', angle)
    return angle

In [281]:
arrs = []
for i in range(1, 332):
    places, rots, size = input_helpers.read_labels('/home/paperspace/Desktop/converted/car/training/suburu_pulling_to_left/suburu04/labels/' + str(i) + '.pickle')
    tmp = input_helpers.get_boxcorners(places, rots, size)
    print (rots)
    #print (places)
    #print(size)
    #print(tmp)
    arrs.append(abs(get_orientation2(tmp[0]) - rots[0]))

[ 0.018361]
[[ 2.27654099 -0.84734559]
 [ 2.24389696  0.93035471]
 [-2.27654099  0.84734559]
 [-2.24389696 -0.93035471]]
[[ 2.24389696  0.93035471]
 [-2.27654099  0.84734559]
 [-2.24389696 -0.93035471]
 [ 2.27654099 -0.84734559]]
[ 1.77800012  4.52120066  1.77800012  4.52120066]
length of car: 4.5212 width of car: 1.778
[[-2.2606003284454346, -0.8890000581741333], [2.2606003284454346, -0.8890000581741333], [-2.2606003284454346, 0.8890000581741333], [-2.2606003284454346, -0.8890000581741333], [-2.2606003284454346, 0.8890000581741333], [2.2606003284454346, 0.8890000581741333], [2.2606003284454346, -0.8890000581741333], [2.2606003284454346, 0.8890000581741333]]
[[-2.24389694 -0.9303547 ]
 [ 2.27654097 -0.84734561]
 [-2.27654097  0.84734561]
 [-2.24389694 -0.9303547 ]
 [-2.27654097  0.84734561]
 [ 2.24389694  0.9303547 ]
 [ 2.27654097 -0.84734561]
 [ 2.24389694  0.9303547 ]]
[[ 0.9998313   0.01835997]
 [-0.01835997  0.99983138]]
angle_diff: 0.018368910442306697
[ 0.018489]
[[ 2.27664948 -0

length of car: 4.5212 width of car: 1.778
[[-2.2606000900268555, -0.8890000581741333], [2.2606000900268555, -0.8890000581741333], [-2.2606000900268555, 0.8890000581741333], [-2.2606000900268555, -0.8890000581741333], [-2.2606000900268555, 0.8890000581741333], [2.2606000900268555, 0.8890000581741333], [2.2606000900268555, -0.8890000581741333], [2.2606000900268555, 0.8890000581741333]]
[[-2.2439323  -0.93026943]
 [ 2.27650877 -0.84743212]
 [-2.27650877  0.84743212]
 [-2.2439323  -0.93026943]
 [-2.27650877  0.84743212]
 [ 2.2439323   0.93026943]
 [ 2.27650877 -0.84743212]
 [ 2.2439323   0.93026943]]
[[ 0.9998321   0.01832197]
 [-0.01832197  0.99983207]]
angle_diff: 0.01832517308838123
[ 0.017352]
[[ 2.27568483 -0.84964222]
 [ 2.24483442  0.92809016]
 [-2.27568483  0.84964222]
 [-2.24483442 -0.92809016]]
[[ 2.24483442  0.92809016]
 [-2.27568483  0.84964222]
 [-2.24483442 -0.92809016]
 [ 2.27568483 -0.84964222]]
[ 1.77800012  4.52120018  1.77800012  4.52120018]
length of car: 4.5212 width o

 [ 2.16421533 -1.10308897]]
[[-2.16421533  1.10308897]
 [-2.33586597 -0.66660595]
 [ 2.16421533 -1.10308897]
 [ 2.33586597  0.66660595]]
[ 4.5211997   1.77800012  4.5211997   1.77800012]
length of car: 4.5212 width of car: 1.778
[[-2.2605998516082764, -0.8890000581741333], [2.2605998516082764, -0.8890000581741333], [-2.2605998516082764, 0.8890000581741333], [-2.2605998516082764, -0.8890000581741333], [-2.2605998516082764, 0.8890000581741333], [2.2605998516082764, 0.8890000581741333], [2.2605998516082764, -0.8890000581741333], [2.2605998516082764, 0.8890000581741333]]
[[-2.33586598 -0.66660596]
 [ 2.16421536 -1.10308895]
 [-2.16421536  1.10308895]
 [-2.33586598 -0.66660596]
 [-2.16421536  1.10308895]
 [ 2.33586598  0.66660596]
 [ 2.16421536 -1.10308895]
 [ 2.33586598  0.66660596]]
[[ 0.99532903 -0.09654141]
 [ 0.0965414   0.9953289 ]]
angle_diff: 0.09669132323085522
[-0.100105]
[[ 2.33812737  0.65862978]
 [-2.16043782  1.11046898]
 [-2.33812737 -0.65862978]
 [ 2.16043782 -1.11046898]]
[

angle_diff: 0.10966540383122883
[-0.106174]
[[ 2.34208155  0.64442766]
 [-2.15365863  1.12356019]
 [-2.34208155 -0.64442766]
 [ 2.15365863 -1.12356019]]
[[-2.15365863  1.12356019]
 [-2.34208155 -0.64442766]
 [ 2.15365863 -1.12356019]
 [ 2.34208155  0.64442766]]
[ 4.5211997   1.77800012  4.5211997   1.77800012]
length of car: 4.5212 width of car: 1.778
[[-2.2605998516082764, -0.8890000581741333], [2.2605998516082764, -0.8890000581741333], [-2.2605998516082764, 0.8890000581741333], [-2.2605998516082764, -0.8890000581741333], [-2.2605998516082764, 0.8890000581741333], [2.2605998516082764, 0.8890000581741333], [2.2605998516082764, -0.8890000581741333], [2.2605998516082764, 0.8890000581741333]]
[[-2.34208163 -0.64442764]
 [ 2.15365874 -1.12356014]
 [-2.15365874  1.12356014]
 [-2.34208163 -0.64442764]
 [-2.15365874  1.12356014]
 [ 2.34208163  0.64442764]
 [ 2.15365874 -1.12356014]
 [ 2.34208163  0.64442764]]
[[ 0.9943689  -0.10597464]
 [ 0.10597462  0.99436877]]
angle_diff: 0.106173384067903

[ 0.066042]
[[ 2.31434059 -0.73787594]
 [ 2.19700336  1.03624809]
 [-2.31434059  0.73787594]
 [-2.19700336 -1.03624809]]
[[ 2.19700336  1.03624809]
 [-2.31434059  0.73787594]
 [-2.19700336 -1.03624809]
 [ 2.31434059 -0.73787594]]
[ 1.778       4.52120018  1.778       4.52120018]
length of car: 4.5212 width of car: 1.778
[[-2.2606000900268555, -0.88899999856948853], [2.2606000900268555, -0.88899999856948853], [-2.2606000900268555, 0.88899999856948853], [-2.2606000900268555, -0.88899999856948853], [-2.2606000900268555, 0.88899999856948853], [2.2606000900268555, 0.88899999856948853], [2.2606000900268555, -0.88899999856948853], [2.2606000900268555, 0.88899999856948853]]
[[-2.19700327 -1.03624804]
 [ 2.3143406  -0.73787595]
 [-2.3143406   0.73787595]
 [-2.19700327 -1.03624804]
 [-2.3143406   0.73787595]
 [ 2.19700327  1.03624804]
 [ 2.3143406  -0.73787595]
 [ 2.19700327  1.03624804]]
[[ 0.99781998  0.065994  ]
 [-0.065994    0.99782002]]
angle_diff: 0.06604260213531986
[ 0.064794]
[[ 2.3134

[ 0.087142]
[[ 2.32939339 -0.68888277]
 [ 2.17465091  1.08237076]
 [-2.32939339  0.68888277]
 [-2.17465091 -1.08237076]]
[[ 2.17465091  1.08237076]
 [-2.32939339  0.68888277]
 [-2.17465091 -1.08237076]
 [ 2.32939339 -0.68888277]]
[ 1.77800024  4.52120018  1.77800024  4.52120018]
length of car: 4.5212 width of car: 1.778
[[-2.2606000900268555, -0.88900011777877808], [2.2606000900268555, -0.88900011777877808], [-2.2606000900268555, 0.88900011777877808], [-2.2606000900268555, -0.88900011777877808], [-2.2606000900268555, 0.88900011777877808], [2.2606000900268555, 0.88900011777877808], [2.2606000900268555, -0.88900011777877808], [2.2606000900268555, 0.88900011777877808]]
[[-2.17465101 -1.0823707 ]
 [ 2.32939347 -0.68888274]
 [-2.32939347  0.68888274]
 [-2.17465101 -1.0823707 ]
 [-2.32939347  0.68888274]
 [ 2.17465101  1.0823707 ]
 [ 2.32939347 -0.68888274]
 [ 2.17465101  1.0823707 ]]
[[ 0.9962055   0.08703175]
 [-0.08703174  0.99620541]]
angle_diff: 0.08714245584629149
[ 0.08672]
[[ 2.32910

[[ 0.99628566  0.08610923]
 [-0.08610923  0.9962857 ]]
angle_diff: 0.08621646076700594
[ 0.086041]
[[ 2.32863355 -0.69144696]
 [ 2.17584133  1.07997572]
 [-2.32863355  0.69144696]
 [-2.17584133 -1.07997572]]
[[ 2.17584133  1.07997572]
 [-2.32863355  0.69144696]
 [-2.17584133 -1.07997572]
 [ 2.32863355 -0.69144696]]
[ 1.77799988  4.5211997   1.77799988  4.5211997 ]
length of car: 4.5212 width of car: 1.778
[[-2.2605998516082764, -0.88899993896484375], [2.2605998516082764, -0.88899993896484375], [-2.2605998516082764, 0.88899993896484375], [-2.2605998516082764, -0.88899993896484375], [-2.2605998516082764, 0.88899993896484375], [2.2605998516082764, 0.88899993896484375], [2.2605998516082764, -0.88899993896484375], [2.2605998516082764, 0.88899993896484375]]
[[-2.17584138 -1.07997576]
 [ 2.3286336  -0.69144699]
 [-2.3286336   0.69144699]
 [-2.17584138 -1.07997576]
 [-2.3286336   0.69144699]
 [ 2.17584138  1.07997576]
 [ 2.3286336  -0.69144699]
 [ 2.17584138  1.07997576]]
[[ 0.99630082  0.0859

In [285]:
print (np.degrees(np.sum(arrs) / len(arrs)))
print (np.degrees(np.median(arrs)))

3.81049608835
0.000130783346409


In [283]:
print(max(arrs))

0.342974229948


In [288]:
from collections import defaultdict

def make_frame_result_dict_shell():
    return {'pred': {}, 'actual': {}}
results = defaultdict(make_frame_result_dict_shell)

In [291]:
#ftest = '/home/paperspace/Desktop/didi_solution/cnn_3d/results_suburu_leading_front_left_conf99_epoch6.pickle'
ftest = '/home/paperspace/Desktop/didi_solution/cnn_3d/results_nissan_driving_past_it_conf95_epoch6.pickle'
import os
print(os.path.exists(ftest))
print(input_helpers.read_labels(ftest))
places, rots, size = input_helpers.read_labels(ftest)
tmp = input_helpers.get_boxcorners(places, rots, size)
print (rots)
print(get_orientation2(tmp[0]))

True


ValueError: too many values to unpack (expected 3)